<a href="https://colab.research.google.com/github/sulthonarifimadudin/uasDL/blob/main/XSum/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ------------------------------------------------------------------
# BAGIAN 1: INSTALL DEPENDENCIES (KHUSUS QLORA)
# ------------------------------------------------------------------
# Kita butuh library khusus untuk menangani model besar & kuantisasi 4-bit
!pip install -q -U torch transformers datasets peft bitsandbytes trl accelerate

import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer # Library khusus training LLM

# Cek GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Config
MODEL_NAME = "microsoft/phi-2"
DATASET_NAME = "xsum"
NEW_MODEL_NAME = "finetuning-phi-2-text-summarization" # Sesuai request



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170

In [ ]:
# ------------------------------------------------------------------
# BAGIAN 2: LOAD DATASET (Official Parquet Branch)
# ------------------------------------------------------------------
print("\n--- Loading Dataset XSum (Official Parquet Revision) ---")

# TRIK RAHASIA:
# Kita akses branch khusus "refs/convert/parquet" dari repo aslinya.
# Ini mem-bypass script .py yang error dan langsung ambil data tabelnya.
dataset = load_dataset(
    "EdinburghNLP/xsum",
    revision="refs/convert/parquet"
)

# Ambil subset kecil untuk demo
SAMPLE_SIZE = 1000
train_dataset = dataset["train"].shuffle(seed=42).select(range(SAMPLE_SIZE))
eval_dataset = dataset["validation"].shuffle(seed=42).select(range(50))

print(f"Train Size: {len(train_dataset)}")

# Cek kolom (Harusnya aman: 'document' dan 'summary')
print(f"Nama Kolom: {train_dataset.column_names}")
print(f"Sample Data:\n{train_dataset[0]}")


--- Loading Dataset XSum (Official Parquet Revision) ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


default/train/0000.parquet:   0%|          | 0.00/304M [00:00<?, ?B/s]

default/validation/0000.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

default/test/0000.parquet:   0%|          | 0.00/17.0M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Train Size: 1000
Nama Kolom: ['document', 'summary', 'id']
Sample Data:
{'document': 'In Wales, councils are responsible for funding and overseeing schools.\nBut in England, Mr Osborne\'s plan will mean local authorities will cease to have a role in providing education.\nAcademies are directly funded by central government and head teachers have more freedom over admissions and to change the way the school works.\nIt is a significant development in the continued divergence of schools systems on either side of Offa\'s Dyke.\nAnd although the Welsh Government will get extra cash to match the money for English schools to extend the school day, it can spend it on any devolved policy area.\nMinisters have no plans to follow suit.\nAt the moment, governing bodies are responsible for setting school hours and they need ministerial permission to make significant changes.\nThere are already more than 2,000 secondary academies in England and its extension to all state schools is unlikely to shake 

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer

# ==========================================
# 1. KONFIGURASI NAMA MODEL
# ==========================================
# Pastikan nama model dasar benar (biasanya microsoft/phi-2)
MODEL_NAME = "microsoft/phi-2"
NEW_MODEL_NAME = "phi2-summarizer-fixed"

# ==========================================
# 2. LOAD MODEL & TOKENIZER (4-BIT)
# ==========================================
print("--- Loading Model & Tokenizer ---")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Safer for training
# Kita naikkan ke 1024 atau 2048 biar berita panjang gak kepotong
tokenizer.model_max_length = 2048 # Changed from 1024 to 2048

# Persiapan Model untuk Training Low-Bit
model = prepare_model_for_kbit_training(model)

# ==========================================
# 3. CONFIG LORA (CRUCIAL FIX!)
# ==========================================
# Target modules ini disesuaikan dengan struktur Phi-2 terbaru
# Biar model beneran belajar dan loss turun drastis.
peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "dense", "fc1", "fc2"]
)

# ==========================================
# 4. FORMAT DATASET (CRUCIAL FIX!)
# ==========================================
# Menggunakan kolom 'document' dan 'summary'
def format_instruction(sample):
    intro = "Summarize the following text:"
    context = sample['document'] # Kolom berita
    response = sample['summary'] # Kolom ringkasan

    # Format Alpaca-style + EOS Token
    return f"### Instruction:\n{intro}\n{context}\n\n### Response:\n{response}{tokenizer.eos_token}"

# ==========================================
# 5. TRAINING ARGUMENTS
# ==========================================
training_args = TrainingArguments(
    output_dir=NEW_MODEL_NAME,
    num_train_epochs=1,
    per_device_train_batch_size=1, # Kecil aja biar gak OOM
    gradient_accumulation_steps=4, # Akumulasi biar stabil
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,       # WAJIB True buat T4
    bf16=False,      # WAJIB False buat T4
    max_grad_norm=0.3,
    logging_steps=10,
    optim="paged_adamw_32bit",
    save_strategy="epoch",
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)

print("--- Initializing Trainer ---")
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    # eval_dataset=eval_dataset, # Uncomment jika ada
    peft_config=peft_config,
    processing_class=tokenizer,
    args=training_args,
    formatting_func=format_instruction
    # max_seq_length=1024, # Sesuaikan dengan tokenizer  <-- REMOVED THIS LINE
    # packing=False,
)

# ==========================================
# 6. THE FIX FOR T4 GPU (Force Float32)
# ==========================================
# Ini obat manjur mengatasi error "Attempting to unscale FP16 gradients"
print("\n--- 🩹 Applying T4 Compatibility Patch ---")
count_fixed = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        # Paksa parameter LoRA jadi Float32
        param.data = param.data.to(torch.float32)
        count_fixed += 1

# LayerNorm juga sebaiknya Float32
for name, module in model.named_modules():
    if "norm" in name.lower():
        module.to(torch.float32)

print(f"✅ Sukses mengubah {count_fixed} parameter LoRA ke Float32.")
print("Model siap tempur!")

# ==========================================
# 7. MULAI TRAINING
# ==========================================
print("\n--- 🚀 Starting Training ---")
# Gunakan resume_from_checkpoint=False untuk memastikan start fresh
trainer.train(resume_from_checkpoint=False)

# ==========================================
# 8. SIMPAN MODEL
# ==========================================
print("\n--- 💾 Saving Model ---")
trainer.model.save_pretrained(NEW_MODEL_NAME)
tokenizer.save_pretrained(NEW_MODEL_NAME)
print("✅ SELESAI! Alhamdulillah.")

--- Loading Model & Tokenizer ---


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

--- Initializing Trainer ---


Applying formatting function to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

...

wandb: You chose "Don't visualize my results"


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.411500
20,2.316800
30,2.247500
40,2.249400
50,2.174800
60,2.429700
70,2.328200
80,2.289000
90,2.182600
100,2.133700


wandb: WARNING URL not available in offline run



--- 💾 Saving Model ---
✅ SELESAI! Alhamdulillah.


In [ ]:
# Cek isi data baris pertama
print("--- ISI DATA BARIS PERTAMA ---")
print(train_dataset[0])

# Cek daftar nama kolom yang tersedia
print("\n--- NAMA KOLOM ---")
print(train_dataset.column_names)

--- ISI DATA BARIS PERTAMA ---
{'document': 'In Wales, councils are responsible for funding and overseeing schools.\nBut in England, Mr Osborne\'s plan will mean local authorities will cease to have a role in providing education.\nAcademies are directly funded by central government and head teachers have more freedom over admissions and to change the way the school works.\nIt is a significant development in the continued divergence of schools systems on either side of Offa\'s Dyke.\nAnd although the Welsh Government will get extra cash to match the money for English schools to extend the school day, it can spend it on any devolved policy area.\nMinisters have no plans to follow suit.\nAt the moment, governing bodies are responsible for setting school hours and they need ministerial permission to make significant changes.\nThere are already more than 2,000 secondary academies in England and its extension to all state schools is unlikely to shake the Welsh Government\'s attachment to wha

In [ ]:
# Pastikan model di mode evaluasi dulu (PENTING!)
model.eval()

def test_model(text_input):
    # Prompt tanpa jawaban
    prompt = f"### Instruction:\nSummarize the following text:\n{text_input}\n\n### Response:\n"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Settingan biar gak halu (Temperature 0.5 lebih stabil buat ringkasan)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,      # Jangan panjang-panjang dulu
            do_sample=True,
            temperature=0.5,         # Agak kaku dikit biar akurat
            top_p=0.9,
            repetition_penalty=1.2,  # Anti ngulang kata
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode hasilnya
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "### Response:" in result:
        return result.split("### Response:\n")[1].strip()
    return result

# --- AMBIL CONTOH DARI DATASET ---
# Kita ambil data ke-10 (random aja)
idx = 10
sample_text = train_dataset[idx]['document']
sample_summary = train_dataset[idx]['summary']

print("=== BERITA ASLI (Potongan) ===")
print(sample_text[:500] + "...")
print("\n=== RINGKASAN ASLI (Kunci Jawaban) ===")
print(sample_summary)
print("\n=== PREDIKSI MODEL (Loss 1.8) ===")
print(test_model(sample_text))

=== BERITA ASLI (Potongan) ===
Ethiopia has not commented on the reported fighting in the Tsorona area, about half-way along the frontier.
Residents on the Ethiopian side of the border reported hearing gunfire and seeing a large movement of troops and artillery towards the border.
A peace deal in 2000 ended the countries' two-year war but it has not been fully implemented.
Ever since, the countries have been in a state of "neither war nor peace", says the BBC's Ethiopia correspondent Emmanuel Igunza.
Eritrea says the tense re...

=== RINGKASAN ASLI (Kunci Jawaban) ===
Eritrea has accused Ethiopia of launching an attack at the countries' heavily-militarised border.

=== PREDIKSI MODEL (Loss 1.8) ===
An apparent clash between soldiers from neighbouring Ethiopia and Eritrea has left one person dead near their shared border.
